In [1]:
from bs4 import BeautifulSoup #for webscraping
import requests               #to get the webpage
import re                     #for regular expression matching

base_url = 'https://pitchfork.com'

#use this page to get the urls for all the best album lists 
page_name = base_url + '/features/lists-and-guides/'
#request the page
page = requests.get(page_name)
#make the soup
soup = BeautifulSoup(page.content)

In [2]:
#2017 list added manually
lists = ['/features/lists-and-guides/the-50-best-albums-of-2017/']

#get all links for 50 best albums of the year
#matching for the regular expression -50-
for link in soup.findAll('a', attrs={'href':re.compile(r'-50-')}):
    lists.append(link['href'])

#remove duplicates
lists = list(set(lists))

In [3]:
def get_album_titles(webpage):
    """
    quick helper function to extract album titles from 
    pitchfork best album of the year lists
    @webpage is the page where the albums are listed
    returns a list of album titles
    """
    page = requests.get(webpage) #request the page
    soup = BeautifulSoup(page.content) #make the soup
    #extract the album titles
    albums = soup.findAll('h2', {'class' : 'list-blurb__work-title'})
    #add the album titles to the list
    return([t.get_text() for t in albums])

#initalize list to store info
album_title = [] 

#best album link years
years = range(2013, 2019)
#for each year
for year in years:
    #filter our list of links for the year
    new_page = base_url + [l for l in lists if str(year) in l][0]
    album_title += get_album_titles(new_page)
    #pitchfork uses a separate web page for every 10 albums
    for next_page in range(2, 6):
        #get the next page
        next_page = new_page + '?page=%s' % next_page
        album_title += get_album_titles(next_page)

In [4]:
import pandas as pd #to create a data frame

#create a one column df using the album titles
album_df = pd.DataFrame({'album_title' : album_title})
#add the year and rank for each album
album_df['year'] = [year for year in years for i in range(50)]
album_df['rank'] = (list(range(50, 0, -1)) * len(years))
#save as a csv file
album_df.to_csv("data/pitchfork_50.csv")

In [39]:
import sqlite3 #to import data from the data base
from functools import reduce #help joining data frames

con = sqlite3.connect("data/pitchfork.db")
#get review data
reviews = pd.read_sql('SELECT * FROM reviews', con) 
#get genre, label, content (text of review)
genres = pd.read_sql('SELECT * FROM genres', con)
labels = pd.read_sql('SELECT * FROM labels', con)
content = pd.read_sql('SELECT * FROM content', con)
con.close()

#get word count from review
content['word_count'] = content['content'].apply(lambda x: len(x.split()))

#data frames to join
dfs = [reviews, genres, labels, content]
#join and create one data frame
reviews_final = reduce(lambda left, right: pd.merge(left, right, on='reviewid'), dfs)
#save as a csv file
reviews_final.to_csv("data/pitchfork_reviews.csv")

In [52]:
#join the top 50 lists with the review data
#review album titles are all lower case
album_df['album_title_lower'] = album_df['album_title'].str.lower()
final_album_df = pd.merge(reviews_final, album_df,  
                          how='left', 
                          left_on= ['title', 'pub_year'], 
                          right_on = ['album_title_lower', 'year'])
final_album_df.to_csv("data/pitchfork50_with_reviews.csv")